In [1]:
import re
import pandas as pd
import unicodedata

#本地归一化函数，保证所有文本格式一样
def normalize_text(text):
    #将输入强制转换成字符串，防止出现NaN或数字导致程序崩溃
    text = str(text)

    #运行unicode标注化(NFKC模式)
    #把全角字符转半角，统一不同编码形式的字符
    text = unicodedata.normalize("NFKC", text)

    #使用正则把多个连续空格(空格,tab,换行)转换成一个空格
    text = re.sub(r"\s+", " ", text)

    #去掉字符串首尾空格，防止出现“看起来一样但其实不同”的文本
    return text.strip()

#定义数据清洗主函数
def clean_dataset(df, min_len=5, max_len=512):
    #对DataFrame中的text列使用本地归一化函数
    #确保所有样本格式统一，减少分布噪声
    df["text"] = df["text"].apply(normalize_text)

    #新建一列length，用于记录每条文本的字符长度
    #用于后续过滤过短或过长的字符样本
    df["length"] = df["text"].apply(len)

    #过滤长度小于或大于阈值的样本
    #过短的文本信息量极低，容易引入标签噪声
    #过长的文本容易导致异常数据或对模型训练不友好
    df = df[df["length"] >= min_len]
    df = df[df["length"] <= max_len]

    #根据text列去重
    df = df.drop_duplicates(subset="text")

    #删除中间计算用的length列，返回干净的数据集
    return df.drop(columns=["length"])

train/ val/ test划分原理：在训练集上拟合参数，在验证集上调超参数，在测试集上模拟真实上线结果

如果不用val，就会在测试集上反复调用模型，test变成“伪训练集”，最终测试指标严重乐观偏估。

| 数据量      | 推荐比例         |
| -------- | ------------ |
| 小数据（<1万） | 80 / 10 / 10 |
| 中等       | 85 / 10 / 5  |
| 大规模      | 90 / 5 / 5   |


NLP数据集必须分层采样，如果随机采样，可能会出现train里90%的正类，val / test全是负类，模型直接失效。

> 正确的采样方式：按 label 分层采样（stratified split）

In [2]:
from sklearn.model_selection import train_test_split
import json
import os

#数据集划分函数
#参数df：清理后的完整数据集
#参数output_dir: 保存train / val / test的文件目录
def split_dataset(df, output_dir):
    #从完整数据集中划分出训练集和临时数据集，临时数据集占比20%
    train_df, temp_df = train_test_split(
        df,
        test_size=0.2,
        stratify=["label"],
        random_state=42
    )

    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.2,
        stratify=["label"],
        random_state=42
    )

    #创建输出目录，exist_ok=True表述目录存在时不会报错
    os.makedirs(output_dir, exist_ok=True)

    #将训练集保存为csv文件，index=False表示不会保存DataFrame的行索引
    train_df.to_csv(f"{output_dir}/train.csv", index=False)

    val_df.to_csv(f"{output_dir}/val.csv", index=False)
    test_df.to_csv(f"{output_dir}/test.csv", index=False)

    #构建一个字典，用于统计各数据集中标签分布的情况
    #value_counts()用于统计每个label的样本数量
    #to_dict()转换成普通发python字典，便于序列化
    stats = {
        "train": train_df["label"].value_counts().to_dict(),
        "val": val_df["label"].value_counts().to_dict(),
        "test": test_df["label"].value_counts().to_dict()
    }

    #以写入模式打开stats.jons文件，
    #使用with可以在写完后自动关闭文件
    with open(f"{output_dir}/stats.jons", "w") as f:
        #indent=2用于美化输出，方便人工查看和调试
        json.dump(stats, f, indent=2)

In [ ]:
import pandas as pd
import os

raw_path = "data/raw/raw_data.csv"
processed_path = "data/processed"
split_path = "data/splits"

def main():
    df = pd.read_csv(raw_path)
    df = clean_dataset(df)

    os.makedirs(processed_path, exist_ok=True)
    df.to_csv(f"{processed_path}/cleaned.csv", index=False)

    split_dataset(df, split_path)

if __name__ == "__main__":
    main()

读取txt文件

In [ ]:
import os

def read_txt_file(path):
    # 以utf-8编码读取文本文件
    with open(path, "r", encodings="utf-8", errors="ignore") as f:
        text = f.read(path)

    return text

读取word文件

In [ ]:
from docx import Document

def read_docx_file(path):
    #加载docx文档
    doc = Document(path)

    #提取所有非空段落，.strip()去除首尾空格、换行符等无意义的字符，保证段落内容时“干净文本”，避免只含空白的噪声数据
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]

    #用换行符拼接成完整文本
    return "\n".join(paragraphs)

把文件转化成结构化样本表

In [7]:
import pandas as pd

#定义一个函数，从指定目录中读取文件，并构建文本分类用的DataFrame
#data_dir: 存放文本文件的目录路径
#label：该目录下所有文本对应的分类标签
def build_dataframe_from_files(data_dir, label):

    #初始化一个空列表，用于储存所有样本
    #每个元素最终是一个字典，{"text": ..., "label": ..., "source_file": ...}
    samples = []

    for filename in os.listdir(data_dir):
        #拼接出文件的完整路径
        file_path = os.path.join(data_dir, filename)

        if filename.endswith(".txt"):
            text = read_txt_file(file_path)

        elif filename.endswith(".docx"):
            text = read_docx_file(file_path)

        else:
            continue

        #按段落切分文本
        #使用换行符把整篇文本拆分成多个段落，这样做可以把“一个文件”拆分成“多个训练样本”
        for paragraph in text.split("\n"):
            #去掉首尾空白字符，并判断长度，去除长度过短的段落，避免噪声
            if len(paragraph.strip()) < 5:
                continue

            #将清洗后的段落加入samples列表
            samples.append({
                #当前段落文本作为模型的输入
                "text": paragraph.strip(),
                "label": label,
                #记录该段落来自哪个原始文件
                #非常重要：后续去重、排查数据泄露、错误分析
                "source_file": filename
            })

    #将文本转换成pandas DataFrame
    #每一行是一个文本样本，每一列是一个特征字段
    return pd.DataFrame(samples)

先按文件分类，再展开样本

文件列表

→ train_files / val_files / test_files

→ 再从各自文件中抽样本

In [ ]:
from sklearn.model_selection import train_test_split

#先对文件名划分
#统计有多少源文件
files = df["source_file"].unique()

train_file, temp_file = train_test_split(
    files,
    test_size=0.2,
    random_state=42
)

val_file, test_file = train_test_split(
    temp_file,
    test_size=0.5,
    random_state=42
)

#再按文件名筛选样本
trian_df = df[df["source_file"]].isin(train_file)
val_df = df[df["source_file"]].isin(val_file)
test_df = df[df["source_file"]].isin(test_file)

如何防止数据泄露？

>第一是样本级泄漏，所有文本在划分前会进行去重，确保同一语义样本不会同时出现在训练集和测试集中；第二是统计级泄漏，像词表构建、TF-IDF、归一化参数只基于训练集计算，再应用到验证集和测试集；第三是特征级泄漏，会人工检查是否存在显式标签提示词，比如评分、标签字段或规则生成文本；第四是分布泄漏，通过分层采样保证各数据集类别分布和真实业务场景一致。